# Get and Upload your on checkpoint to SambaStudio

In [ ]:
# Import libraries
import os
import sys
import re
import json
import time
from huggingface_hub import hf_hub_download, HfApi

# Get absolute paths for kit_dir and repo_dir
current_dir = os.getcwd()
kit_dir =  os.path.abspath(os.path.join(current_dir, '..'))
repo_dir = os.path.abspath(os.path.join(kit_dir, '..'))

# Adding directories to the Python module search path
sys.path.append(repo_dir)

from utils.dedicated_env.snsdk_wrapper import SnsdkWrapper

/Users/jorgep/Documents/ask_public_own/snapienv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step by Step / Manual setting

First instantiate the SambaStudio client for BYOC 

In [ ]:
snsdk_wrapper = SnsdkWrapper()

2024-11-25 12:20:51,796 [INFO] Using variables from Snapi config to set up Snsdk.


### Download a base chekpoint from HuggingFace (Optional)

You can use your own fine-tuned models or alternatively you can download and use [Huggingface model checkpoints](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending). In this case we will use an available model in HuggingFace 

In [3]:
hf_model = 'lightblue/suzume-llama-3-8B-multilingual'
target_dir = os.path.join(kit_dir, 'data', 'models') 

In [16]:
# Create target dir if not exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Download checkpoint to target dir
repo_files = HfApi().list_repo_files(hf_model)
for file_name in repo_files:
    hf_hub_download(repo_id=hf_model, filename=file_name, cache_dir=target_dir)

In [4]:
# Find the snapshot folder inside the target dir
for root, dirs, files in os.walk(target_dir):
    if "snapshots" in root and hf_model.replace("/", "--") in root:
        checkpoint_folder = os.path.join(root,dirs[0])
        break

checkpoint_folder

'/Users/jorgep/Documents/ask_public_own/ai-starter-kit-snova/e2e_fine_tuning/data/models/models--lightblue--suzume-llama-3-8B-multilingual/snapshots/0cb15aa9ec685eef494f9a15f65aefcfe3c04c66'

### Set checkpoint configs

Some parameter should be provided to upload a previously created checkpoint, for this we will keep these parameters in a checkpoint dict 

In [5]:
checkpoint = {
    'model_name':'Suzume-Llama-3-8B-Multilingual',
    'publisher': "lightblue",
    'description': " Suzume 8B, a multilingual finetune of Llama 3",
    'param_count':8,  # number in billions of parameters
    'checkpoint_path': checkpoint_folder
}

### Set and check chat template (Optional) 

If you want to use chat templates (roles structures), you could need to include or update the existing chat template. This should be formatted as a Jinja2 String template as the following example

In [17]:
jinja_chat_template = """ 
{% for message in messages %}
    {% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>' + '\n' + message['content'] | trim + '<|eot_id|>'+'\n' %}
    {% if loop.index0 == 0 %}{% set content = bos_token + content %}
    {% endif %}
    {{content}}
{% endfor %}
{{'<|start_header_id|>assistant<|end_header_id|>'+'\n'}}
"""
#delete scape characters
jinja_chat_template = re.sub(r"(?<!')\n(?!')", "", jinja_chat_template).strip().replace('  ','')

In [ ]:
# Open and update current tokenizer config from the checkpoint path
with open(os.path.join(checkpoint['checkpoint_path'], 'tokenizer_config.json'), 'r+') as file:
    data = json.load(file)
    data['chat_template'] = jinja_chat_template
    file.seek(0)
    file.truncate()
    json.dump(data, file, indent=4)

In [ ]:
# Render template when using a roles / chat structure
test_messages = [
    {"role": "system", "content": "This is a system prompt."},
    {"role": "user", "content": "This is a user prompt."},
    {"role": "assistant", "content": "This is a response from the assistant."},
    {"role": "user", "content": "This is an user follow up"}
    ]
snsdk_wrapper.check_chat_templates(test_messages, checkpoint_paths=checkpoint['checkpoint_path'])

2024-11-25 15:17:29,527 [INFO] Raw chat template for checkpoint in /Users/jorgep/Documents/ask_public_own/ai-starter-kit-snova/e2e_fine_tuning/data/models/models--lightblue--suzume-llama-3-8B-multilingual/snapshots/0cb15aa9ec685eef494f9a15f65aefcfe3c04c66:
{% for message in messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>' + '
' + message['content'] | trim + '<|eot_id|>'+'
' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{content}}{% endfor %}{{'<|start_header_id|>assistant<|end_header_id|>'+'
'}}

2024-11-25 15:17:29,532 [INFO] Rendered template with input test messages:

<|begin_of_text|><|start_header_id|>system<|end_header_id|>
This is a system prompt.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
This is a user prompt.<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
This is a response from the assistant.<|eot_id|>
<|start_header_id|>user<|end_header_id|>
This is an user follow up<|eot_id|>
<|start_

### Set padding token (required for training)

if not set in your checkpoint and you want to do a further fine-tuning over your checkpoint you need to ensure the 'pad_token_id' is set in your checkpoint `config.json`

In [19]:
# Adding pad_token_id to checkpoint config
with open(os.path.join(checkpoint['checkpoint_path'], 'config.json'), 'r+') as file:
    data = json.load(file)
    data['pad_token_id']=None
    file.seek(0)
    file.truncate()
    json.dump(data, file, indent=4)

### Get model params and Sambastudio suitable Apps

Extra parameters are required to upload your checkpoint including model architecture, sequence length, and vocabulary size, those parameters can be extracted from your checkpoint config, and included in checkpoint dict parameters


In [ ]:
checkpoint_config_params = snsdk_wrapper.find_config_params(checkpoint_paths=checkpoint['checkpoint_path'])[0]
checkpoint.update(checkpoint_config_params)

2024-11-25 12:21:33,276 [INFO] Params for checkpoint in /Users/jorgep/Documents/ask_public_own/ai-starter-kit-snova/e2e_fine_tuning/data/models/models--lightblue--suzume-llama-3-8B-multilingual/snapshots/0cb15aa9ec685eef494f9a15f65aefcfe3c04c66:
[{'model_arch': 'llama', 'seq_length': 8192, 'vocab_size': 128256}]


To upload a model checkpoint you will need to set a SambaStudio App, if not sure, you can search for suitable apps using the checkpoint parameters, and select the one from the suitable apps

In [ ]:
suitable_apps = snsdk_wrapper.get_suitable_apps(checkpoint)

2024-11-25 12:21:35,336 [INFO] Checkpoint Suzume-Llama-3-8B-Multilingual suitable apps:
[{'id': '61fa0993-04a2-42ca-9db1-1eff693ea978', 'name': 'Samba1 Llama3 Experts'}, {'id': 'ad39e323-9878-4914-8e29-82c9f2939475', 'name': 'Llama 3'}]


In [ ]:
# from the two suitable apps found we will use the first one give we want to use this checkpoint in a bundle
checkpoint["app_id"]=suitable_apps[0][0]['id']

In [13]:
# We can see here all the parameters required to upload the checkpoint
checkpoint

{'model_name': 'Suzume-Llama-3-8B-Multilingual',
 'publisher': 'lightblue',
 'description': ' Suzume 8B, a multilingual finetune of Llama 3',
 'param_count': 8,
 'checkpoint_path': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit-snova/e2e_fine_tuning/data/models/models--lightblue--suzume-llama-3-8B-multilingual/snapshots/0cb15aa9ec685eef494f9a15f65aefcfe3c04c66',
 'model_arch': 'llama',
 'seq_length': 8192,
 'vocab_size': 128256,
 'app_id': '61fa0993-04a2-42ca-9db1-1eff693ea978'}

### Upload the checkpoint

If uploading only one checkpoint

In [ ]:
# Execute the upload_checkpoint method from client with checkpoint parameters (this can take a while)
model_id=snsdk_wrapper.upload_checkpoint(
    model_name=checkpoint['model_name'],
    checkpoint_path=checkpoint['checkpoint_path'],
    description=checkpoint['description'],
    publisher=checkpoint['publisher'],
    param_count=checkpoint['param_count'],
    model_arch=checkpoint['model_arch'],
    seq_length=checkpoint['seq_length'],
    vocab_size=checkpoint['vocab_size'],
    app_id=checkpoint['app_id'],
    retries=3
)

2024-11-25 12:21:42,947 [INFO] Model with name 'Suzume-Llama-3-8B-Multilingual' not found
2024-11-25 12:21:43,138 [INFO] App with name '61fa0993-04a2-42ca-9db1-1eff693ea978' found with id 61fa0993-04a2-42ca-9db1-1eff693ea978
2024-11-25 15:17:27,207 [INFO] Model with name 'Suzume-Llama-3-8B-Multilingual' found with id b2bb7c02-3b18-4ef4-bd58-1b19669bce15
2024-11-25 15:17:27,208 [INFO] Model checkpoint with name 'Suzume-Llama-3-8B-Multilingual' created it with id b2bb7c02-3b18-4ef4-bd58-1b19669bce15


In [ ]:
# check the status of the uploaded checkpoint 
snsdk_wrapper.get_checkpoints_status(model_id)

2024-11-25 15:36:10,969 [INFO] model b2bb7c02-3b18-4ef4-bd58-1b19669bce15 status: 
 {'model_id': 'b2bb7c02-3b18-4ef4-bd58-1b19669bce15', 'status': 'Available', 'progress': 100, 'stage': 'convert', 'status_code': 200, 'headers': {'access-control-allow-headers': 'Accept, Content-Type, Content-Length, Accept-Encoding, Authorization, ResponseType', 'access-control-allow-methods': 'GET, POST, PATCH, DELETE', 'access-control-allow-origin': '', 'content-type': 'application/json', 'x-correlation-id': 'cd512837-7f7d-48f6-a9df-62b6ef3e0d69', 'date': 'Mon, 25 Nov 2024 20:36:11 GMT', 'x-envoy-upstream-service-time': '52', 'server': 'istio-envoy', 'content-encoding': 'gzip', 'vary': 'Accept-Encoding', 'transfer-encoding': 'chunked'}}


Alternatively if uploading multiple checkpoints in parallel

In [ ]:
models=snsdk_wrapper.upload_checkpoints([checkpoint], max_parallel_jobs=4, retries=3) # add multiple checkpoints parameters to the checkpoints list
snsdk_wrapper.get_checkpoints_status([model['id'] for model in models])

## Streamlined Execution
The checkpoint upload can be done in a streamlined way setting all the checkpoints parameters to upload in a config file like in the [checkpoints_config.yaml](../checkpoints_config.yaml) example, and executing:

In [ ]:
config_file = os.path.join(kit_dir, 'checkpoints_config.yaml')
snsdk_wrapper = SnsdkWrapper(config_file)
snsdk_wrapper.find_config_params()
snsdk_wrapper.upload_checkpoints()
# wait until all checkpoints are in available status
while True:
    statuses = [model['status'] for model in snsdk_wrapper.get_checkpoints_status()]
    if all(x == "Available" for x in statuses):
        break
    else:
        time.sleep(10)